1) A. Lakukan eksplorasi data terlebih dahulu untuk memahami permasalahan 
yang dihadapi terlebih dahulu. Dataset yang diberikan adalah data time series, lakukan 
praproses data untuk menyelesaikan problem dari data tersebut. Pisahkan data time series 
tersebut menjadi dua bagian input dan output dengan window size = 5 [dari hari senin s.d 
jumat] dan horizon = 1 [hari senin saja]. Selanjutnya pisahkan dataset menjadi train, test dan 
validation set dengan ketentuan (80 train, 10 val, 10 test) 

In [2]:
#Load data

import numpy as np
import pandas as pd

amzn_data = pd.read_csv('D://AMZN.csv')
csco_data = pd.read_csv('D://CSCO.csv')

#Hanya menggunakan column date and close
amzn_data = amzn_data[['Date', 'Close']]
csco_data = csco_data[['Date', 'Close']]

amzn_data['Date'] = pd.to_datetime(amzn_data['Date'])
csco_data['Date'] = pd.to_datetime(csco_data['Date'])

amzn_data = amzn_data.sort_values(by='Date').reset_index(drop=True)
csco_data = csco_data.sort_values(by='Date').reset_index(drop=True)

In [3]:
amzn_data.head()

,Date,Close
0,1997-05-15,1.958333
1,1997-05-16,1.729167
2,1997-05-19,1.708333
3,1997-05-20,1.635417
4,1997-05-21,1.427083


In [4]:
csco_data.head()

,Date,Close
0,1990-02-16,0.077257
1,1990-02-20,0.079861
2,1990-02-21,0.078125
3,1990-02-22,0.078993
4,1990-02-23,0.078559


In [7]:
#Membuat window size = 5

def create_windows(data, window_size, horizon):
    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data.iloc[i:(i + window_size)].values)
        y.append(data.iloc[(i + window_size):(i + window_size + horizon)].values)
    return np.array(X), np.array(y)

window_size = 5
horizon = 1
X_amzn, y_amzn = create_windows(amzn_data['Close'], window_size, horizon)
X_csco, y_csco = create_windows(csco_data['Close'], window_size, horizon)

In [8]:
#Splitting data menjadi train, test dan validation set dengan ketentuan (80 train, 10 val, 10 test) 

def train_val_test_split(X, y, train_size=0.8, val_size=0.1, test_size=0.1):
    assert train_size + val_size + test_size == 1
    train_end = int(len(X) * train_size)
    val_end = int(len(X) * (train_size + val_size))
    X_train, X_val, X_test = X[:train_end], X[train_end:val_end], X[val_end:]
    y_train, y_val, y_test = y[:train_end], y[train_end:val_end], y[val_end:]
    return X_train, X_val, X_test, y_train, y_val, y_test

X_train_amzn, X_val_amzn, X_test_amzn, y_train_amzn, y_val_amzn, y_test_amzn = train_val_test_split(X_amzn, y_amzn)
X_train_csco, X_val_csco, X_test_csco, y_train_csco, y_val_csco, y_test_csco = train_val_test_split(X_csco, y_csco)

X_train_amzn.shape, X_val_amzn.shape, X_test_amzn.shape, y_train_amzn.shape, y_val_amzn.shape, y_test_amzn.shape
X_train_csco.shape, X_val_csco.shape, X_test_csco.shape, y_train_csco.shape, y_val_csco.shape, y_test_csco.shape


((6067, 5), (758, 5), (759, 5), (6067, 1), (758, 1), (759, 1))

1) B. Buatlah arsitektur baseline dengan LSTM (units=50) dan layer akhir berupa 
node Perceptron dengan units=1. Activation function untuk LSTM menggunakan ReLU 

In [9]:
#Reshape data

def correct_reshape(X):
    return X.reshape((X.shape[0], X.shape[1], 1))

X_train_amzn = correct_reshape(X_train_amzn)
X_val_amzn = correct_reshape(X_val_amzn)
X_test_amzn = correct_reshape(X_test_amzn)

X_train_csco = correct_reshape(X_train_csco)
X_val_csco = correct_reshape(X_val_csco)
X_test_csco = correct_reshape(X_test_csco)

In [7]:
#Arsitektur baseline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model1 = Sequential()
model1.add(LSTM(units=50, activation='relu', input_shape=(window_size, 1)))
model1.add(Dense(units=1))

model1.compile(optimizer='adam', loss='mse')
model1.summary()

history_amzn = model1.fit(X_train_amzn, y_train_amzn, epochs=20, validation_data=(X_val_amzn, y_val_amzn))
history_csco = model1.fit(X_train_csco, y_train_csco, epochs=20, validation_data=(X_val_csco, y_val_csco))

D:\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 50)                  │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 6869.7017 - val_loss: 476.5648
Epoch 2/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.6343 - val_loss: 437.7341
Epoch 3/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.9524 - val_loss: 841.2777
Epoch 4/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.2080 - val_loss: 1444.4231
Epoch 5/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 24.0602 - val_loss: 389.0662
Epoch 6/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 28.1714 - val_loss: 755.8729
Epoch 7/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 27.6154 - val_loss: 544.3539
Epoch 8/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 24.3441 - val_loss: 499.7581
Epoch 9/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24.0139 - val_loss: 315.9945
Epoch 10/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 24.4666 - val_loss: 314.3519
Epoch 11/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 26.5901 - val_loss: 365.6304
Epoch 12/20
144/

In [8]:
y_pred_amzn_1 = model1.predict(X_test_amzn)
y_pred_csco_1 = model1.predict(X_test_csco)


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


1) C.Setelah mengetahui hasil dari nomor (1c), modifikasi arsitektur 
pada nomor 1c untuk mendapatkan unjuk kerja yang optimal (kalian dapat menambahkan atau 
mengurangi arsitektur tersebut, atau mengganti hyperparameter, atau menggunakan tuning 
pada hyperparameter). Jelaskan alasan kalian untuk menggunakan pendekatan yang kalian pilih.

ANSWER
Setelah melihat hasil pada baseline model, saya hanya memodifikasi sedikit arsitekturnya dengan mengganti activation function dari ReLU menjadi tanh. Selebihnya sama. Lalu saya juga menambahkan learning rate sebesar 0.0010 pada arsitektur yang sudah dimodifikasi. Alasan saya menggunakan pendekatan ini adalah: 
- Saya sudah mencoba menambahkan 1 layer, 2layer, dsb serta menambahkan dropout layer sebesar 0.3, 0.4 dan 0.5. Akan tetapi dengan melakukan penambahan tersebut hasil dari model semakin buruk dengan angka loss dan val loss yang malah bertambah tinggi. 
- Saya juga sudah mencoba menggunakakn relu saat menambahkan layer & dropout tapi tetap saja hasilnya semakin buruk. 
- Selain itu untuk angka learning rate saya memilih 0.0010 karena saya sudah mencoba angka lain seperti 0.01, 0.005, 0.001 akan tetapi berdasarkan hasilnya 0.0010 memberikan hasil yang paling baik. 

In [10]:
#Reshape data

def reshape_data(X):
    return X.reshape((X.shape[0], X.shape[1], 1))

X_train_amzn = reshape_data(X_train_amzn)
X_val_amzn = reshape_data(X_val_amzn)
X_test_amzn = reshape_data(X_test_amzn)

X_train_csco = reshape_data(X_train_csco)
X_val_csco = reshape_data(X_val_csco)
X_test_csco = reshape_data(X_test_csco)

In [9]:
#Modified model

model2 = Sequential()
model2.add(LSTM(units=50, activation='tanh', input_shape=(window_size, 1)))
model2.add(Dense(units=1))

model2.compile(optimizer=Adam(learning_rate=0.0010), loss='mse')
model2.summary()

history_amzn = model2.fit(X_train_amzn, y_train_amzn, epochs=20, validation_data=(X_val_amzn, y_val_amzn))
history_csco = model2.fit(X_train_csco, y_train_csco, epochs=20, validation_data=(X_val_csco, y_val_csco))

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 50)                  │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 24928.3984 - val_loss: 638420.6875
Epoch 2/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 20426.4512 - val_loss: 623783.6250
Epoch 3/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 19519.2910 - val_loss: 612923.1875
Epoch 4/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 18642.1953 - val_loss: 602902.3750
Epoch 5/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 17828.4121 - val_loss: 593403.6875
Epoch 6/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 17129.4219 - val_loss: 584255.1875
Epoch 7/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 16164.9082 - val_loss: 575468.8750
Epoch 8/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 15021.3057 - val_loss: 566832.5000
Epoch 9/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 13593.8389 - val_loss: 558451.0625
Epoch 10/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 13520.8965 - val_loss: 550197.4375
Epoch 11/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/s

In [10]:
y_pred_amzn_2 = model2.predict(X_test_amzn)
y_pred_csco_2 = model2.predict(X_test_csco)


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


1) D.Lakukan evaluasi unjuk kerja kedua arsitektur di atas pada test set dengan 
mencari nilai RMSE, MAE dan MAPE. Dan berikan penjelasan mengenai hasilnya dengan rinci. 

In [11]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def evaluate_performance(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    return rmse, mae, mape

In [13]:
#Evaluation

#Model1
rmse_amzn_1, mae_amzn_1, mape_amzn_1 = evaluate_performance(y_test_amzn, y_pred_amzn_1)
rmse_csco_1, mae_csco_1, mape_csco_1 = evaluate_performance(y_test_csco, y_pred_csco_1)

#Model2
rmse_amzn_2, mae_amzn_2, mape_amzn_2 = evaluate_performance(y_test_amzn, y_pred_amzn_2)
rmse_csco_2, mae_csco_2, mape_csco_2 = evaluate_performance(y_test_csco, y_pred_csco_2)

#Result
print(f"First Model (LSTM ReLU):")
print(f"AMZN - RMSE: {rmse_amzn_1}, MAE: {mae_amzn_1}, MAPE: {mape_amzn_1}")
print(f"CSCO - RMSE: {rmse_csco_1}, MAE: {mae_csco_1}, MAPE: {mape_csco_1}")

print(f"\nSecond Model (LSTM tanh):")
print(f"AMZN - RMSE: {rmse_amzn_2}, MAE: {mae_amzn_2}, MAPE: {mape_amzn_2}")
print(f"CSCO - RMSE: {rmse_csco_2}, MAE: {mae_csco_2}, MAPE: {mape_csco_2}")

First Model (LSTM ReLU):
AMZN - RMSE: 52.971680201585556, MAE: 42.045628017849396, MAPE: 2.4418587493866313
CSCO - RMSE: 0.8757505646688019, MAE: 0.6122162709594244, MAPE: 1.4299722972725792

Second Model (LSTM tanh):
AMZN - RMSE: 1620.0698427470043, MAE: 1608.4415191411972, MAPE: 92.85633714179079
CSCO - RMSE: 0.8075588052300229, MAE: 0.5336914741003465, MAPE: 1.2332477880682915


Analysis dari evaluasi: Keduanya di run sebanyak 20 epochs. 
Dapat disimpulkan dari hasil epoch dan hasil evaluasi di atas bahwa: Baik untuk CSCO, tetapi buruk untuk AMZN. 
- Model 1: 
Model 1 yang merupakan baseline model using relu memberikan hasil loss cukup tinggi untuk AMZN yaitu >10000. Sedangkan untuk CSCO memberikan hasil yang cukup baik yaitu loss <1. Hal ini berarti model1 membuat prediksi yang cukup akurat untuk CSCO dan prediksi yang cukup buruk untuk AMZN. Jika dilihat dari evaluasinya juga mendukung statement sebelumnya. 
- Model 2: 
Untuk Model 2 juga kurang lebih sama seperti model 1, yaitu baik untuk CSCO, tetapi buruk untuk AMZN. Bahkan dengan menggunakan model2 RMSE, MAE, dan MAPE AMZN lebih tinggi lagi. Tetapi untuk CSCO sudah lebih baik dari model sebelumnya. 
- Hasil loss yang naik turun dapat disebabkan karena data yang kompleks dan hyperparemeter tuning yang belum optimal. 
- Analisis metrics: 
AMZN RMSE 52.9 MAE 42.04
CSCO RMSE 0.88 MAE 0.61
Hal ini berarti bahwa prediksi pada model CSCO lebih baik dibandingkan AMZN serta tingkat kesalahan prediksi untuk CSCO lebih rendah dibandingkan AMZN. 

Dengan demikian dapat saya simpulkan bahwa mungkin yang menyebabkan AMZN memiliki prediksi yang buruk dari kedua model adalah karena datanya overfit seta data yang lebih fluktuatif dibandingkan data CSCO. Hal ini menyebabkan model yang saya buat sulit untuk mempelajari data dan membuat prediksi. 